In [ ]:
#START HERE#
#Open Libraries - Run every time
library(mnormt)
library(Seurat)
library(ggplot2)
library(dplyr)
library(cowplot)
library(reticulate)
set.seed(42)

# Loading scRNA-seq quantification results

## Specify the folder containing the sparse matrix files

In [ ]:
wti.data   <- Read10X(data.dir ="/Users/yzhou14/Data_local/20191010_scRNAseq_analysis/Raw_Data/WTI/")
wtc.data   <- Read10X(data.dir ="/Users/yzhou14/Data_local/20191010_scRNAseq_analysis/Raw_Data/WTC/")
c3koi.data <- Read10X(data.dir ="/Users/yzhou14/Data_local/20191010_scRNAseq_analysis/Raw_Data/C3KOI/")
c3koc.data <- Read10X(data.dir ="/Users/yzhou14/Data_local/20191010_scRNAseq_analysis/Raw_Data/C3KOC/")

## Create Seurat object and drop low quality cell

In [ ]:
#Set up first object
wti <- CreateSeuratObject(counts = wti.data, project = "WILD_TYPE", min.cells = 3)
wti[["percent.mt"]] <- PercentageFeatureSet(wti, pattern = "^mt-")
wti <- subset(wti, subset = nFeature_RNA > 200 & nFeature_RNA < 2500 & percent.mt < 25)
wti$stim <- "WTI"
#wti <- SCTransform(wti, vars.to.regress = "percent.mt", verbose = TRUE)

# Set up second object
wtc <- CreateSeuratObject(counts = wtc.data, project = "WILD_TYPE", min.cells = 3)
wtc[["percent.mt"]] <- PercentageFeatureSet(wtc, pattern = "^mt-")
wtc <- subset(wtc, subset = nFeature_RNA > 200 & nFeature_RNA < 2500 & percent.mt < 25)
wtc$stim <- "WTC"
#wtc <- SCTransform(wtc, vars.to.regress = "percent.mt", verbose = TRUE)

# Set up third object
c3koi <- CreateSeuratObject(counts = c3koi.data, project = "GEMM", min.cells = 3)
c3koi[["percent.mt"]] <- PercentageFeatureSet(c3koi, pattern = "^mt-")
c3koi <- subset(c3koi, subset = nFeature_RNA > 200 & nFeature_RNA < 2500 & percent.mt < 25)
c3koi$stim <- "C3KOI"
#c3koi <- SCTransform(c3koi, vars.to.regress = "percent.mt", verbose = TRUE)

# Set up fourth object
c3koc <- CreateSeuratObject(counts = c3koc.data, project = "GEMM", min.cells = 3)
c3koc[["percent.mt"]] <- PercentageFeatureSet(c3koc, pattern = "^mt-")
c3koc <- subset(c3koc, subset = nFeature_RNA > 200 & nFeature_RNA < 2500 & percent.mt < 25)
c3koc$stim <- "C3KOC"
#c3koc <- SCTransform(c3koc, vars.to.regress = "percent.mt", verbose = TRUE)


## Integrative preprocessing across multiple groups

In [ ]:
#Integrate objects
anchor.set <- FindIntegrationAnchors(object.list = list(wti, wtc, c3koi, c3koc), dims = 1:20)
immune.combined <- IntegrateData(anchorset = anchor.set, dims = 1:20)
DefaultAssay(immune.combined) <- "integrated"
immune.combined <- SCTransform(immune.combined, vars.to.regress = "percent.mt", verbose = TRUE)
immune.combined <- RunPCA(immune.combined, npcs = 30, verbose = TRUE)

In [ ]:
#Determine appropriate number of PCA
print(immune.combined[["pca"]], dims = 1:16, nfeatures = 16)
ElbowPlot(immune.combined)

# Dimensionality reduction and clustering

## unsupervised clustering

In [ ]:
# Dimensionality reduction and Clustering
immune.combined <- RunUMAP(immune.combined, reduction = "pca", dims = 1:16)
immune.combined <- FindNeighbors(immune.combined, reduction = "pca", dims = 1:16)

#Change resolution to modify number or clusters
immune.combined <- FindClusters(immune.combined, resolution = 0.6)

## Algorithmic cell type definition

In [ ]:
library(SingleR)
immgen <- ImmGenData()

SingleR_Annotation <- function(seu, reference="HPCA", use_local=T){
    if (use_local==F){
        if       (reference=="HPCA"){
            ref <- HumanPrimaryCellAtlasData()   
        }else if (reference=="BED"){
            ref <- BlueprintEncodeData()
        }else if (reference=="DbImmExp"){
            ref <- DatabaseImmuneCellExpressionData()
        }else if (reference=="Hemato"){
            ref <- NovershternHematopoieticData()
        }else if (reference=="Monaco"){
            ref <- MonacoImmuneData()
        }else if (reference=="ImmGen"){
            ref <- ImmGenData()
        }else if (reference=="MouseRNA"){
            ref <- MouseRNAseqData()
        }else{
            ref <- HumanPrimaryCellAtlasData()   
        }
    }else{
        if       (reference=="HPCA"){
            ref <- hpca
        }else if (reference=="BED"){
            ref <- blueprint
        }else if (reference=="DbImmExp"){
            ref <- dbimmexp
        }else if (reference=="Hemato"){
            ref <- hemato
        }else if (reference=="Monaco"){
            ref <- monaco
        }else if (reference=="ImmGen"){
            ref <- immgen
        }else if (reference=="MouseRNA"){
            ref <- mmrna
        }else{
            ref <- hpca
        }
    }
    
        
    mat       <- GetAssayData(seu, slot="data")
    pred.fine <- SingleR(test = mat, ref = ref, labels = ref$label.fine)
    pred.main <- SingleR(test = mat, ref = ref, labels = ref$label.main)
       
    seu[[paste0(reference,"_main")]]<-pred.main$pruned.labels
    seu[[paste0(reference,"_fine")]]<-pred.fine$pruned.labels
    return(seu) 
}

In [ ]:
tic()
DefaultAssay(immune.combined)<-"RNA"
immune.combined<-SingleR_Annotation(seu=immune.combined, reference="ImmGen", use_local=T)
toc()

In [ ]:
options(repr.plot.width=10, repr.plot.height=10)
DimPlot(immune.combined, group.by="celltype0627", pt.size=0.1, label=T,repel=T)

## Adjusting the metadata

In [ ]:
# adjust the stim group information, make it brief
metadata=immune.combined@meta.data
row.select=metadata$stim=="C3KOC"
metadata[row.select, "stim"]="KOC"
row.select=metadata$stim=="C3KOI"
metadata[row.select, "stim"]="KOI"
immune.combined@meta.data=metadata

# Rename cluster id
Idents(immune.combined)<-"SCT_snn_res.0.6"
new.cluster.ids <- c(
  "0.IgD+ mature B", 
  "1.IgM+IgD+ mature B", 
  "2.CD4+CTLA4+ T", 
  "3.CD4+ Teff", 
  "4.Myeloid/monocytic cells", 
  "5.Myeloid phagocytes", 
  "6.IgA+ Plasma B", 
  "7.Gata3+ ILC2,ILC3", 
  "8.RORga+ ILC2,ILC3",
  "9.Foxp3+ Treg", 
  "10.CD8+ T, NKT", 
  "11.IgD+ mature B", 
  "12.Neu",
  "13.proliferare B", 
  "14.B plasmablast",
  "15.CX3CR1+ Mac", 
  "16.Epithelial", 
  "17.Endothelial", 
  "18.Fibroblasts")
names(new.cluster.ids) <- levels(immune.combined)
immune.combined <- RenameIdents(immune.combined, new.cluster.ids)
immune.combined[["celltype0627"]]<-Idents(immune.combined)

# merge the cell type annotation with the stim annotation
immune.combined[["celltype0627_stim"]]<- paste(Idents(immune.combined), immune.combined$stim, sep = "_")

In [ ]:
#Save Clusters for easy retrieval
saveRDS(immune.combined, file = "/Users/yzhou14/Data_local/20200625_scRNAseq_analysis/Colon_0.6_0627.RDS")

In [ ]:
#Read it again elsewheres
immune.combined<-readRDS(file ="/Users/yzhou14/Data_local/20200303_scRNAseq_analysis/Colon_0.6_0627.RDS")